# Introduction with a Mars Express Example

This is a Python Jupyter Notebook to illustrate a SPICE running example. 
This could be your very own Python 3 environment, the first thing we will do is indicate that we want the Python package matplotlib to be output in the notebook and to import the SpiceyPy package to use SPICE.

In [ ]:
%matplotlib inline

import math
import spiceypy as spiceypy

## A Mars Express Phobos observation example

In this first example, we will compute some basic geometry for a Phobos observation by the HRSC camera, more concretely the SRC sensor. We an retrieve the image from the Planetary Science Archive (PSA) User Interface. 

![title](remote_sensing_lesson/img/HF780_0004_SR2.JPG)

From the PDS3 label of the image we obtain:

   * PRODUCT_CREATION_TIME             = 2016-10-12T14:44:59.000Z   

In [ ]:
spiceypy.furnsh('/Users/mcosta/SPICE/MARS-EXPRESS/kernels/mk/MEX_OPS_LOCAL.TM')

We convert from UTC to Ephemeris Time (ET) the time of the image: 2016-06-14T03:11:56.218Z

In [ ]:
et = spiceypy.utc2et('2016-06-14T03:11:56.208')

We obtain the HRSC boresight and boresight reference frame

In [ ]:
sensor_name = 'MEX_HRSC_SRC'

sensor_id = spiceypy.bodn2c(sensor_name)
(shape, frame, bsight, vectors, bounds) = spiceypy.getfov(sensor_id, 100)

print('{} shape:  {}'.format(sensor_name, shape))
print('{} frame:  {}'.format(sensor_name, frame))
print('{} bsight: {}'.format(sensor_name, bsight))

We obtain the intersection between the boresight and Phobos:

In [ ]:
try: 
    (spoint, trgepc, srfvec ) = spiceypy.sincpt('ELLIPSOID', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', frame, bsight)
except:
    print('No intersection found!')

Of course, it does not work because we are currently not intersecting the body, so let's look for an moment in time in which there is interception. We can simply modify our vector in the HRSC SRC frame:

In [ ]:
bsight = [-4.64050347,  0.0, 984.73813222]

try:
    (spoint, trgepc, srfvec ) = spiceypy.sincpt('ELLIPSOID', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', frame, bsight)
except:
    print('No intersection found!')

Finally we compute the illumination angles

In [ ]:
(trgepc, srfvec, phase, solar, emissn) = spiceypy.ilumin('ELLIPSOID', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', spoint)

print('Phase Angle: {:.3f}     [DEG]\nSolar Incidence: {:.3f} [DEG]'.format(math.degrees(phase), math.degrees(solar)))

But we could load a Digital Shape Model of PHOBOS and assess the difference in illumination by using the shape model. The DSK that we load is as follows:

![title](remote_sensing_lesson/img/phobos_dsk.png)

In [ ]:
spiceypy.furnsh('remote_sensing_lesson/kernels/dsk/phobos_k137_dlr_v01.bds')

(spoint, trgepc, srfvec ) = spiceypy.sincpt('DSK/UNPRIORITIZED', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', frame, bsight)
(trgepc, srfvec, phase, solar, emissn) = spiceypy.ilumin('DSK/UNPRIORITIZED', 'PHOBOS', et, 'IAU_PHOBOS', 'NONE', 'MEX', spoint)

print('Phase Angle: {:.3f}     [DEG]\nSolar Incidence: {:.3f} [DEG]'.format(math.degrees(phase), math.degrees(solar)))

And this is what we would see in Cosmographia:
    
![title](remote_sensing_lesson/img/cosmo.png)


Now, we will show the object oriented capabilities of sciops, aimed to easy the way that we interface with SPICE. 
We will define a Time Window, Phobos as a target and Mars Express as an observer:

In [ ]:
import spiops as spiops

interval = spiops.TimeWindow('2016-11-26T00:07:15', '2016-11-26T10:21:32', resolution=10)

phobos = spiops.Target('PHOBOS', time=interval, frame='IAU_PHOBOS')
mex = spiops.Observer('MEX', time=interval, target=phobos)

The spiops library will plot some geometric quantites at our desire:

In [ ]:
mex.Plot('distance', notebook=True)

In [ ]:
spiceypy.kclear()